# Визуализация калибровки LiDAR ↔️ Камера

Этот ноутбук демонстрирует два типичных шага при работе с `BEVCalib`:
1. Как выглядит лидарное облако точек при раскалибровке и после корректной калибровки.
2. Как можно получить проекцию Bird's-Eye View (BEV) как из облака точек, так и из изображения.

## Импорт библиотек и настройки
В примере используются только стандартные библиотеки Python и зависимости, уже имеющиеся в проекте.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 - необходим для 3D-графиков
import cv2

np.random.seed(42)

## Синтетическое облако точек
Чтобы не зависеть от данных KITTI, создадим небольшую синтетическую сцену: плоскость дороги, прямоугольник автомобиля и столб.

In [ ]:
def create_synthetic_scene():
    # Плоскость дороги (решетка точек)
    ground_x = np.linspace(0, 30, 20)
    ground_y = np.linspace(-10, 10, 25)
    gx, gy = np.meshgrid(ground_x, ground_y)
    ground = np.stack([gx.ravel(), gy.ravel(), np.zeros_like(gx).ravel()], axis=1)

    # Параллелепипед автомобиля
    car_x = np.linspace(8, 12, 8)
    car_y = np.linspace(-1.2, 1.2, 6)
    car_z = np.linspace(0, 1.6, 6)
    cx, cy, cz = np.meshgrid(car_x, car_y, car_z)
    car = np.stack([cx, cy, cz], axis=-1).reshape(-1, 3)
    car_shell = car[(np.isclose(car[:, 0], car_x[0])) |
                    (np.isclose(car[:, 0], car_x[-1])) |
                    (np.isclose(car[:, 1], car_y[0])) |
                    (np.isclose(car[:, 1], car_y[-1])) |
                    (np.isclose(car[:, 2], car_z[-1]))]

    # Столб/дерево
    pole_z = np.linspace(0, 4, 30)
    pole = np.stack([np.full_like(pole_z, 18.0),
                     np.full_like(pole_z, 5.0),
                     pole_z], axis=1)

    points = np.vstack([ground, car_shell, pole])
    points += np.random.normal(scale=0.03, size=points.shape)
    return points

points_lidar = create_synthetic_scene()
points_lidar.shape

## Экстринсики и раскалибровка
Определим истинные параметры (вращение + перенос) и добавим шум, имитирующий раскалибровку.

In [ ]:
def rotation_matrix(roll=0.0, pitch=0.0, yaw=0.0):
    cr, sr = np.cos(roll), np.sin(roll)
    cp, sp = np.cos(pitch), np.sin(pitch)
    cy, sy = np.cos(yaw), np.sin(yaw)

    rot_x = np.array([[1, 0, 0],
                      [0, cr, -sr],
                      [0, sr, cr]])
    rot_y = np.array([[cp, 0, sp],
                      [0, 1, 0],
                      [-sp, 0, cp]])
    rot_z = np.array([[cy, -sy, 0],
                      [sy, cy, 0],
                      [0, 0, 1]])
    return rot_z @ rot_y @ rot_x


def make_extrinsic(rotation, translation):
    extrinsic = np.eye(4)
    extrinsic[:3, :3] = rotation
    extrinsic[:3, 3] = translation
    return extrinsic


def apply_extrinsic(points, extrinsic):
    points_h = np.hstack([points, np.ones((points.shape[0], 1))])
    transformed = points_h @ extrinsic.T
    return transformed[:, :3]


true_rotation = rotation_matrix(roll=np.deg2rad(-1.5),
                                pitch=np.deg2rad(1.0),
                                yaw=np.deg2rad(2.5))
true_translation = np.array([0.25, -0.45, 1.75])
true_extrinsic = make_extrinsic(true_rotation, true_translation)

miscal_rotation = rotation_matrix(roll=np.deg2rad(1.2),
                                  pitch=np.deg2rad(-2.0),
                                  yaw=np.deg2rad(4.5))
miscal_translation = true_translation + np.array([0.35, 0.25, -0.18])
miscalibrated_extrinsic = make_extrinsic(miscal_rotation, miscal_translation)

# После калибровки мы стремимся восстановить истинные параметры.
points_in_camera_miscal = apply_extrinsic(points_lidar, miscalibrated_extrinsic)
points_in_camera_calibrated = apply_extrinsic(points_lidar, true_extrinsic)

## Сравнение облаков точек
На графиках ниже показано, как одна и та же сцена выглядит в системе координат камеры до и после калибровки.

In [ ]:
%matplotlib inline


def set_axes_equal(ax):
    limits = np.array([
        ax.get_xlim3d(),
        ax.get_ylim3d(),
        ax.get_zlim3d(),
    ])
    spans = limits[:, 1] - limits[:, 0]
    centers = np.mean(limits, axis=1)
    radius = 0.5 * max(spans)
    ax.set_xlim3d([centers[0] - radius, centers[0] + radius])
    ax.set_ylim3d([centers[1] - radius, centers[1] + radius])
    ax.set_zlim3d([centers[2] - radius, centers[2] + radius])


fig = plt.figure(figsize=(14, 6))
ax1 = fig.add_subplot(121, projection='3d')
ax1.scatter(points_in_camera_miscal[:, 0],
           points_in_camera_miscal[:, 1],
           points_in_camera_miscal[:, 2],
           c='tomato', s=6, alpha=0.7)
ax1.set_title('До калибровки')
ax1.set_xlabel('X (камера)')
ax1.set_ylabel('Y (камера)')
ax1.set_zlabel('Z (камера)')
set_axes_equal(ax1)

ax2 = fig.add_subplot(122, projection='3d')
ax2.scatter(points_in_camera_calibrated[:, 0],
           points_in_camera_calibrated[:, 1],
           points_in_camera_calibrated[:, 2],
           c='royalblue', s=6, alpha=0.7)
ax2.set_title('После калибровки')
ax2.set_xlabel('X (камера)')
ax2.set_ylabel('Y (камера)')
ax2.set_zlabel('Z (камера)')
set_axes_equal(ax2)

plt.tight_layout()
plt.show()

## BEV из облака точек
Для Bird's-Eye View используем простую гистограмму по сетке $x, y$, где значение ячейки показывает количество точек.

In [ ]:
def compute_bev(points, x_limits=(0, 30), y_limits=(-10, 10), resolution=0.2):
    mask = (
        (points[:, 0] >= x_limits[0]) & (points[:, 0] <= x_limits[1]) &
        (points[:, 1] >= y_limits[0]) & (points[:, 1] <= y_limits[1])
    )
    filtered = points[mask]
    x_bins = int(np.ceil((x_limits[1] - x_limits[0]) / resolution))
    y_bins = int(np.ceil((y_limits[1] - y_limits[0]) / resolution))
    bev = np.zeros((y_bins, x_bins), dtype=np.float32)

    x_idx = ((filtered[:, 0] - x_limits[0]) / resolution).astype(int)
    y_idx = ((filtered[:, 1] - y_limits[0]) / resolution).astype(int)
    np.add.at(bev, (y_idx, x_idx), 1)
    bev /= bev.max() + 1e-6
    return bev


bev_from_lidar = compute_bev(points_in_camera_calibrated)

plt.figure(figsize=(6, 6))
plt.imshow(bev_from_lidar, origin='lower', cmap='viridis',
           extent=[0, 30, -10, 10])
plt.colorbar(label='Нормированная плотность точек')
plt.title('BEV из LiDAR (после калибровки)')
plt.xlabel('X, м (вперёд)')
plt.ylabel('Y, м (влево)')
plt.show()

## BEV из изображения
Сымитируем камеру: зададим BEV-карту дороги, спроецируем её в перспективу, а затем восстановим BEV посредством обратного преобразования.

In [ ]:
def create_synthetic_bev(size=400):
    bev = np.zeros((size, size, 3), dtype=np.uint8)
    # Дорога
    cv2.rectangle(bev, (60, 0), (size - 60, size), (80, 80, 80), thickness=-1)
    # Полоса безопасности
    cv2.rectangle(bev, (0, 0), (60, size), (30, 30, 30), thickness=-1)
    cv2.rectangle(bev, (size - 60, 0), (size, size), (30, 30, 30), thickness=-1)
    # Линии разметки
    for offset in [size // 2 - 60, size // 2, size // 2 + 60]:
        for start in range(0, size, 40):
            cv2.rectangle(bev, (offset - 4, start), (offset + 4, start + 20),
                          (230, 230, 230), thickness=-1)
    return bev


bev_template = create_synthetic_bev(400)

# Гомография из BEV в перспективу (имитируем вид фронтальной камеры)
src = np.float32([[0, 399], [399, 399], [320, 220], [80, 220]])
dst = np.float32([[120, 359], [520, 359], [420, 60], [220, 60]])
H = cv2.getPerspectiveTransform(src, dst)

perspective_image = cv2.warpPerspective(bev_template, H, (640, 360))
H_inv = np.linalg.inv(H)
bev_from_image = cv2.warpPerspective(perspective_image, H_inv, (400, 400))

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
axes[0].imshow(cv2.cvtColor(bev_template, cv2.COLOR_BGR2RGB))
axes[0].set_title('Заданная BEV-карта')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(perspective_image, cv2.COLOR_BGR2RGB))
axes[1].set_title('Синтетическое изображение камеры')
axes[1].axis('off')

axes[2].imshow(cv2.cvtColor(bev_from_image, cv2.COLOR_BGR2RGB))
axes[2].set_title('BEV, восстановленное из изображения')
axes[2].axis('off')

plt.show()

## Сопоставление BEV из LiDAR и изображения
Для визуального сравнения можно совместить карты. Ниже показана простая альфа-композиция.

In [ ]:
bev_lidar_color = plt.cm.viridis(bev_from_lidar)[:, :, :3]
bev_lidar_rgb = (bev_lidar_color * 255).astype(np.uint8)

# Приводим размеры к одному разрешению
bev_lidar_resized = cv2.resize(bev_lidar_rgb, (400, 400), interpolation=cv2.INTER_LINEAR)
combined = cv2.addWeighted(bev_lidar_resized, 0.6,
                           cv2.cvtColor(bev_from_image, cv2.COLOR_BGR2RGB), 0.4, 0)

plt.figure(figsize=(6, 6))
plt.imshow(combined)
plt.title('Совмещение BEV из LiDAR и изображения')
plt.axis('off')
plt.show()

## Выводы
- На синтетическом примере видно, как смещения и вращения экстринсиков искажают облако точек и как калибровка их исправляет.
- Простая гистограмма по сетке позволяет получить BEV-представление из LiDAR.
- Гомография демонстрирует принцип получения BEV из изображения — ключевая идея `BEVCalib`.